## Proyecto Final

In [3]:
import pandas as pd 
import numpy as np
import pyodbc
import os

## Convertir CSV a Parquet y Guardarlo

## ETL archivo 2017PurchasePicesDec

In [4]:
from google.cloud import storage

In [5]:
import os
from google.cloud import storage

# Establecer la ruta correcta de las credenciales
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\\Proyecto Final Python\\arcane-geode-449913-m8-2f7230c7d0b6.json"

# Probar la autenticación
try:
    client = storage.Client()
    buckets = list(client.list_buckets())
    print("✅ Autenticación exitosa. Buckets disponibles:", [bucket.name for bucket in buckets])
except Exception as e:
    print("❌ Error de autenticación:", e)

✅ Autenticación exitosa. Buckets disponibles: ['licoreradatos']


In [6]:
def cargar_csv(ruta_csv):
    """Carga un archivo CSV en un DataFrame de Pandas."""
    df = pd.read_csv(ruta_csv)
    print(f"Archivo cargado: {ruta_csv}")
    return df

def revisar_datos(df):
    """Revisa el total de filas y columnas, valores faltantes y duplicados."""
    print("\n🔍 Revisión inicial de los datos:")
    print(f"Forma del DataFrame (filas, columnas): {df.shape}")
    print("\nValores faltantes por columna:\n", df.isnull().sum())
    
    duplicados = df.duplicated().sum()
    print(f"\nTotal de filas duplicadas: {duplicados}")

def limpiar_datos(df):
    """Limpia los datos eliminando valores nulos y 'unknown' en Size y Volume."""
    
    # Eliminar filas con valores nulos
    df = df.dropna()
    
    # Filtrar filas con "unknown" en Size o Volume
    unknown_mask = df["Size"].str.contains("unknown", case=False, na=False) | \
                   df["Volume"].astype(str).str.contains("unknown", case=False, na=False)
    
    df_clean = df[~unknown_mask]
    
    print(f"\nDatos después de limpieza: {df_clean.shape}")
    return df_clean

def renombrar_columnas(df):
    """Renombra las columnas del DataFrame según un formato estándar."""
    df.columns = [
        "ProductoId", "NombreProducto","TipoLicor", "Precio", "Tamano", "Volumen",
        "Clasificacion", "Costo", "ProveedorId", "NombreProveedor"
    ]
    print("\n📌 Columnas renombradas correctamente.")
    return df

def guardar_parquet(df, output_path):
    """Guarda el DataFrame en formato Parquet con compresión Snappy."""
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df.to_parquet(output_path, engine="pyarrow", compression="snappy")
    print(f"\n✅ Archivo guardado en Parquet: {output_path}")


# ------------------ 🔹 EJECUCIÓN DEL PROCESO ETL 🔹 ------------------
ruta_csv = r"C:\\Proyecto Final Henry\\DataBase\\2017PurchasePricesDec1.csv"
ruta_parquet = r"C:\\Proyecto Final Henry\\DataBase\\processed\\Productos.parquet"

# 1️⃣ Cargar datos
df = cargar_csv(ruta_csv)

# 2️⃣ Revisar datos
revisar_datos(df)

# 3️⃣ Limpiar datos
df = limpiar_datos(df)

# 4️⃣ Renombrar columnas
df = renombrar_columnas(df)

# 5️⃣ Guardar en Parquet
guardar_parquet(df, ruta_parquet)

Archivo cargado: C:\\Proyecto Final Henry\\DataBase\\2017PurchasePricesDec1.csv

🔍 Revisión inicial de los datos:
Forma del DataFrame (filas, columnas): (12261, 10)

Valores faltantes por columna:
 Brand             0
Description       1
Tipo de Licor     1
Price             0
Size              1
Volume            1
Classification    0
PurchasePrice     0
VendorNumber      0
VendorName        0
dtype: int64

Total de filas duplicadas: 0

Datos después de limpieza: (12256, 10)

📌 Columnas renombradas correctamente.

✅ Archivo guardado en Parquet: C:\\Proyecto Final Henry\\DataBase\\processed\\Productos.parquet


In [7]:
import pandas as pd

# Cargar un archivo Parquet
df = pd.read_parquet("C:\\Proyecto Final Henry\\DataBase\\processed\\Productos.parquet")

# Mostrar las primeras filas
print(df.head())

print(df.columns)

   ProductoId               NombreProducto           TipoLicor  Precio Tamano  \
0          58  Gekkeikan Black & Gold Sake  Sake/ Shochu/ Soju   12.99  750mL   
1          62     Herradura Silver Tequila             Tequila   36.99  750mL   
2          63   Herradura Reposado Tequila             Tequila   38.99  750mL   
3          72         No. 3 London Dry Gin             Ginebra   34.99  750mL   
4          75    Three Olives Tomato Vodka               Vodka   14.99  750mL   

  Volumen  Clasificacion  Costo  ProveedorId              NombreProveedor  
0     750              1   9.28         8320  SHAW ROSS INT L IMP LTD      
1     750              1  28.67         1128  BROWN-FORMAN CORP            
2     750              1  30.46         1128  BROWN-FORMAN CORP            
3     750              1  26.11         9165  ULTRA BEVERAGE COMPANY LLP   
4     750              1  10.94         7245  PROXIMO SPIRITS INC.         
Index(['ProductoId', 'NombreProducto', 'TipoLicor', 'Prec

### Subir archivo al bucket a la carpeta Datos Limpios

In [8]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\\Proyecto Final Python\\arcane-geode-449913-m8-2f7230c7d0b6.json"

In [9]:
import os
print(os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))

C:\\Proyecto Final Python\\arcane-geode-449913-m8-2f7230c7d0b6.json


In [10]:
def subir_a_gcs(bucket_name, source_file_path, folder_name, destination_file_name):
    """Sube un archivo a una carpeta específica dentro de un bucket en Google Cloud Storage."""
    
    # Inicializar el cliente de GCS
    storage_client = storage.Client()
    
    # Obtener el bucket
    bucket = storage_client.bucket(bucket_name)
    
    # Definir la ruta completa en el bucket (carpeta + nombre de archivo)
    destination_blob_name = f"{folder_name}/{destination_file_name}".strip("/")
    
    # Crear un blob (objeto en el bucket)
    blob = bucket.blob(destination_blob_name)
    
    # Subir el archivo
    blob.upload_from_filename(source_file_path)

    print(f"\n🚀 Archivo {source_file_path} subido a gs://{bucket_name}/{destination_blob_name}")

# Datos específicos de tu bucket y archivo
bucket_name = "licoreradatos"  # Tu bucket en GCS
folder_name = "Datos Limpios"  # Carpeta dentro del bucket
source_file_path = r"C:\\Proyecto Final Henry\\DataBase\\processed\\Productos.parquet"  # Archivo local
destination_file_name = "Productos.parquet"  # Nombre final en el bucket

# Subir archivo a GCS
subir_a_gcs(bucket_name, source_file_path, folder_name, destination_file_name)


🚀 Archivo C:\\Proyecto Final Henry\\DataBase\\processed\\Productos.parquet subido a gs://licoreradatos/Datos Limpios/Productos.parquet


## ETL archivo BegInvFINAL12312026

In [11]:
def cargar_csv(ruta_csv1):
    """Carga un archivo CSV en un DataFrame de Pandas."""
    df1 = pd.read_csv(ruta_csv1)
    print(f"Archivo cargado: {ruta_csv1}")
    return df1

def revisar_datos(df1):
    """Revisa el total de filas y columnas, valores faltantes y duplicados."""
    print("\n🔍 Revisión inicial de los datos:")
    print(f"Forma del DataFrame (filas, columnas): {df1.shape}")
    print("\nValores faltantes por columna:\n", df1.isnull().sum())
    
    duplicados = df1.duplicated().sum()
    print(f"\nTotal de filas duplicadas: {duplicados}")

def renombrar_columnas(df1):
    """Renombra las columnas del DataFrame según un formato estándar."""
    df1.columns = [
        "InventarioInicialId", "Tienda", "Ciudad", "ProductoId", "NombreProducto",
        "Tamano", "Cantidad", "Precio", "FechaInicio", "Costo"
    ]
    print("\n📌 Columnas renombradas correctamente.")
    return df1  # ¡Asegúrate de devolver el DataFrame!

def guardar_parquet(df1, output_path):
    """Guarda el DataFrame en formato Parquet con compresión Snappy."""
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df1.to_parquet(output_path, engine="pyarrow", compression="snappy")
    print(f"\n✅ Archivo guardado en Parquet: {output_path}")

# ------------------ 🔹 EJECUCIÓN DEL PROCESO ETL 🔹 ------------------
ruta_csv1 = r"C:\\Proyecto Final Henry\\DataBase\\BegInvFINAL123120161.csv"
ruta_parquet1 = r"C:\\Proyecto Final Henry\\DataBase\\processed\\InventarioInicial.parquet"

# 1️⃣ Cargar datos
df1 = cargar_csv(ruta_csv1)

# 2️⃣ Revisar datos
revisar_datos(df1)

# 3️⃣ Renombrar columnas
df1 = renombrar_columnas(df1)

# 4️⃣ Guardar en Parquet
guardar_parquet(df1, ruta_parquet1)

Archivo cargado: C:\\Proyecto Final Henry\\DataBase\\BegInvFINAL123120161.csv

🔍 Revisión inicial de los datos:
Forma del DataFrame (filas, columnas): (206529, 10)

Valores faltantes por columna:
 InventoryId      0
Store            0
City             0
Brand            0
Description      0
Size             0
onHand           0
Price            0
startDate        0
PurchasePrice    0
dtype: int64

Total de filas duplicadas: 0

📌 Columnas renombradas correctamente.

✅ Archivo guardado en Parquet: C:\\Proyecto Final Henry\\DataBase\\processed\\InventarioInicial.parquet


In [12]:
import pandas as pd

# Cargar un archivo Parquet
df = pd.read_parquet("C:\\Proyecto Final Henry\\DataBase\\processed\\InventarioInicial.parquet")

# Mostrar las primeras filas
print(df.head())

print(df.columns)

  InventarioInicialId  Tienda        Ciudad  ProductoId  \
0   1_HARDERSFIELD_58       1  HARDERSFIELD          58   
1   1_HARDERSFIELD_60       1  HARDERSFIELD          60   
2   1_HARDERSFIELD_62       1  HARDERSFIELD          62   
3   1_HARDERSFIELD_63       1  HARDERSFIELD          63   
4   1_HARDERSFIELD_72       1  HARDERSFIELD          72   

                NombreProducto Tamano  Cantidad  Precio FechaInicio  Costo  
0  Gekkeikan Black & Gold Sake  750mL         8   12.99    1/1/2016   9.28  
1       Canadian Club 1858 VAP  750mL         7   10.99    1/1/2016   7.40  
2     Herradura Silver Tequila  750mL         6   36.99    1/1/2016  28.67  
3   Herradura Reposado Tequila  750mL         3   38.99    1/1/2016  30.46  
4         No. 3 London Dry Gin  750mL         6   34.99    1/1/2016  26.11  
Index(['InventarioInicialId', 'Tienda', 'Ciudad', 'ProductoId',
       'NombreProducto', 'Tamano', 'Cantidad', 'Precio', 'FechaInicio',
       'Costo'],
      dtype='object')


### Subir archivo al bucket a la carpeta Datos Limpios

In [13]:
def subir_a_gcs(bucket_name, source_file_path1, folder_name, destination_file_name1):
    """Sube un archivo a una carpeta específica dentro de un bucket en Google Cloud Storage."""
    
    # Inicializar el cliente de GCS
    storage_client = storage.Client()
    
    # Obtener el bucket
    bucket = storage_client.bucket(bucket_name)
    
    # Definir la ruta completa en el bucket (carpeta + nombre de archivo)
    destination_blob_name = f"{folder_name}/{destination_file_name1}".strip("/")
    
    # Crear un blob (objeto en el bucket)
    blob = bucket.blob(destination_blob_name)
    
    # Subir el archivo
    blob.upload_from_filename(source_file_path1)

    print(f"\n🚀 Archivo {source_file_path1} subido a gs://{bucket_name}/{destination_blob_name}")

# Datos específicos de tu bucket y archivo
bucket_name = "licoreradatos"  # Tu bucket en GCS
folder_name = "Datos Limpios"  # Carpeta dentro del bucket
source_file_path1 = r"C:\\Proyecto Final Henry\\DataBase\\processed\\InventarioInicial.parquet"  # Archivo local
destination_file_name1 = "InventarioInicial.parquet"  # Nombre final en el bucket

# Subir archivo a GCS
subir_a_gcs(bucket_name, source_file_path1, folder_name, destination_file_name1)


🚀 Archivo C:\\Proyecto Final Henry\\DataBase\\processed\\InventarioInicial.parquet subido a gs://licoreradatos/Datos Limpios/InventarioInicial.parquet


## ETL archivo EndInvFINAL12312026

In [14]:
def cargar_csv(ruta_csv3):
    """Carga un archivo CSV en un DataFrame de Pandas."""
    df3 = pd.read_csv(ruta_csv3)
    print(f"✅ Archivo cargado: {ruta_csv3}")
    return df3

def revisar_datos(df3):
    """Revisa el total de filas y columnas, valores faltantes y duplicados."""
    print("\n🔍 Revisión inicial de los datos:")
    print(f"Forma del DataFrame (filas, columnas): {df3.shape}")
    print("\nValores faltantes por columna:\n", df3.isnull().sum())
    
    duplicados = df3.duplicated().sum()
    print(f"\nTotal de filas duplicadas: {duplicados}")

def replace_nan_city(df3, store_number, city_name):
    """Reemplaza los valores NaN en 'City' con el nombre de la ciudad si Store es el especificado."""
    df3.loc[(df3['Store'] == store_number) & (df3['City'].isna()), 'City'] = city_name
    return df3

def limpiar_datos(df3):
    """Limpia los datos eliminando valores nulos y reemplazando valores faltantes en City."""
    df3 = replace_nan_city(df3, 46, "TYWARDREATH")  # ✅ Corrige valores de City
    return df3

def renombrar_columnas(df3):
    """Renombra las columnas en el orden especificado."""
    columnas_originales = df3.columns.tolist()
    
    nombres_nuevos = [
        "InventarioFinalId", "Tienda", "Ciudad", "ProductoId", "NombreProducto",
        "Tamano", "Cantidad", "Precio", "FechaFin", "Costo"
    ]
    
    # Si el número de columnas no coincide, se evita el error
    if len(columnas_originales) != len(nombres_nuevos):
        print("⚠️ Advertencia: La cantidad de columnas no coincide con los nombres nuevos.")
        print(f"Columnas originales: {columnas_originales}")
        return df3  # Retorna sin cambios

    df3.columns = nombres_nuevos  # ✅ Se renombraron las columnas correctamente
    print("\n📌 Columnas renombradas correctamente.")
    return df3

def guardar_parquet(df3, output_path):
    """Guarda el DataFrame en formato Parquet con compresión Snappy."""
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df3.to_parquet(output_path, engine="pyarrow", compression="snappy")
    print(f"\n✅ Archivo guardado en Parquet: {output_path}")

# ------------------ 🔹 EJECUCIÓN DEL PROCESO ETL 🔹 ------------------
ruta_csv3 = r"C:\\Proyecto Final Henry\\DataBase\\EndInvFINAL123120161.csv"
ruta_parquet3 = r"C:\\Proyecto Final Henry\\DataBase\\processed\\InventarioFinal.parquet"

# 1️⃣ Cargar datos
df3 = cargar_csv(ruta_csv3)

# 2️⃣ Revisar datos
revisar_datos(df3)

# 3️⃣ Limpiar datos
df3 = limpiar_datos(df3)  # ✅ Se asegura de que df3 no sea None

# 4️⃣ Renombrar columnas antes de guardar
df3 = renombrar_columnas(df3)

# 5️⃣ Guardar en Parquet
guardar_parquet(df3, ruta_parquet3)  # ✅ Ahora df3 es válido y no dará error


✅ Archivo cargado: C:\\Proyecto Final Henry\\DataBase\\EndInvFINAL123120161.csv

🔍 Revisión inicial de los datos:
Forma del DataFrame (filas, columnas): (224489, 10)

Valores faltantes por columna:
 InventoryId         0
Store               0
City             1284
Brand               0
Description         0
Size                0
onHand              0
Price               0
endDate             0
PurchasePrice       0
dtype: int64

Total de filas duplicadas: 0

📌 Columnas renombradas correctamente.

✅ Archivo guardado en Parquet: C:\\Proyecto Final Henry\\DataBase\\processed\\InventarioFinal.parquet


In [15]:
import pandas as pd

# Cargar un archivo Parquet
df = pd.read_parquet("C:\\Proyecto Final Henry\\DataBase\\processed\\InventarioFinal.parquet")

# Mostrar las primeras filas
print(df.head())

print(df.columns)

   InventarioFinalId  Tienda        Ciudad  ProductoId  \
0  1_HARDERSFIELD_58       1  HARDERSFIELD          58   
1  1_HARDERSFIELD_62       1  HARDERSFIELD          62   
2  1_HARDERSFIELD_63       1  HARDERSFIELD          63   
3  1_HARDERSFIELD_72       1  HARDERSFIELD          72   
4  1_HARDERSFIELD_75       1  HARDERSFIELD          75   

                NombreProducto Tamano  Cantidad  Precio    FechaFin  Costo  
0  Gekkeikan Black & Gold Sake  750mL        11   12.99  12/31/2016   9.28  
1     Herradura Silver Tequila  750mL         7   36.99  12/31/2016  28.67  
2   Herradura Reposado Tequila  750mL         7   38.99  12/31/2016  30.46  
3         No. 3 London Dry Gin  750mL         4   34.99  12/31/2016  26.11  
4    Three Olives Tomato Vodka  750mL         7   14.99  12/31/2016  10.94  
Index(['InventarioFinalId', 'Tienda', 'Ciudad', 'ProductoId', 'NombreProducto',
       'Tamano', 'Cantidad', 'Precio', 'FechaFin', 'Costo'],
      dtype='object')


### Subir archivo al bucket a la carpeta Datos Limpios

In [16]:
def subir_a_gcs(bucket_name, source_file_path3, folder_name, destination_file_name3):
    """Sube un archivo a una carpeta específica dentro de un bucket en Google Cloud Storage."""
    
    # Inicializar el cliente de GCS
    storage_client = storage.Client()
    
    # Obtener el bucket
    bucket = storage_client.bucket(bucket_name)
    
    # Definir la ruta completa en el bucket (carpeta + nombre de archivo)
    destination_blob_name = f"{folder_name}/{destination_file_name3}".strip("/")
    
    # Crear un blob (objeto en el bucket)
    blob = bucket.blob(destination_blob_name)
    
    # Subir el archivo
    blob.upload_from_filename(source_file_path3)

    print(f"\n🚀 Archivo {source_file_path3} subido a gs://{bucket_name}/{destination_blob_name}")

# Datos específicos de tu bucket y archivo
bucket_name = "licoreradatos"  # Tu bucket en GCS
folder_name = "Datos Limpios"  # Carpeta dentro del bucket
source_file_path3 = r"C:\\Proyecto Final Henry\\DataBase\\processed\\InventarioFinal.parquet"  # Archivo local
destination_file_name3 = "InventarioFinal.parquet"  # Nombre final en el bucket

# Subir archivo a GCS
subir_a_gcs(bucket_name, source_file_path3, folder_name, destination_file_name3)


🚀 Archivo C:\\Proyecto Final Henry\\DataBase\\processed\\InventarioFinal.parquet subido a gs://licoreradatos/Datos Limpios/InventarioFinal.parquet


## ETL archivo InvoicePurchases12312016

In [17]:
def cargar_csv(ruta_csv4):
    """Carga un archivo CSV en un DataFrame de Pandas."""
    df4 = pd.read_csv(ruta_csv4)
    print(f"✅ Archivo cargado: {ruta_csv4}")
    return df4

def revisar_datos(df4):
    """Revisa el total de filas y columnas, valores faltantes y duplicados."""
    print("\n🔍 Revisión inicial de los datos:")
    print(f"Forma del DataFrame (filas, columnas): {df4.shape}")
    print("\nValores faltantes por columna:\n", df4.isnull().sum())
    
    duplicados = df4.duplicated().sum()
    print(f"\nTotal de filas duplicadas: {duplicados}")

    # Mostrar valores únicos en la columna 'Approval' si existe
    if "Approval" in df4.columns:
        unique_values = df4["Approval"].unique()
        print("\n📌 Valores únicos en la columna 'Approval':")
        print(unique_values)

def limpiar_datos(df4):
    """Limpia los datos eliminando valores nulos y la columna Approval."""
    # Eliminar la columna 'Approval' si existe
    if "Approval" in df4.columns:
        df4 = df4.drop(columns=["Approval"])
        print("\n🚮 Columna 'Approval' eliminada.")

    # Eliminar filas con valores nulos
    df4 = df4.dropna()
    
    print("\n✅ Datos limpiados correctamente.")
    return df4  # Asegurar que devuelve el DataFrame limpio

def renombrar_columnas(df4):
    """Renombra las columnas del DataFrame según un formato estándar."""
    df4.columns = [
        "ProveedorId", "NombreProveedor", "FechaFactura", "CompraId", 
        "FechaCompra", "FechaPago", "Cantidad", "CostoTotal", "CostoEnvio"
    ]
    print("\n📌 Columnas renombradas correctamente.")
    return df4  # Asegurar que devuelve el DataFrame actualizado

def guardar_parquet(df4, output_path):
    """Guarda el DataFrame en formato Parquet con compresión Snappy."""
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df4.to_parquet(output_path, engine="pyarrow", compression="snappy")
    print(f"\n✅ Archivo guardado en Parquet: {output_path}")

# ------------------ 🔹 EJECUCIÓN DEL PROCESO ETL 🔹 ------------------
ruta_csv4 = r"C:\\Proyecto Final Henry\\DataBase\\InvoicePurchases12312016.csv"
ruta_parquet4 = r"C:\\Proyecto Final Henry\\DataBase\\processed\\Compras.parquet"

# 1️⃣ Cargar datos
df4 = cargar_csv(ruta_csv4)

# 2️⃣ Revisar datos
revisar_datos(df4)

# 3️⃣ Limpiar datos
df4 = limpiar_datos(df4)

# 4️⃣ Renombrar columnas
df4 = renombrar_columnas(df4)

# 5️⃣ Guardar en Parquet
guardar_parquet(df4, ruta_parquet4)

✅ Archivo cargado: C:\\Proyecto Final Henry\\DataBase\\InvoicePurchases12312016.csv

🔍 Revisión inicial de los datos:
Forma del DataFrame (filas, columnas): (5543, 10)

Valores faltantes por columna:
 VendorNumber       0
VendorName         0
InvoiceDate        0
PONumber           0
PODate             0
PayDate            0
Quantity           0
Dollars            0
Freight            0
Approval        5169
dtype: int64

Total de filas duplicadas: 0

📌 Valores únicos en la columna 'Approval':
[nan 'Frank Delahunt']

🚮 Columna 'Approval' eliminada.

✅ Datos limpiados correctamente.

📌 Columnas renombradas correctamente.

✅ Archivo guardado en Parquet: C:\\Proyecto Final Henry\\DataBase\\processed\\Compras.parquet


In [18]:
import pandas as pd

# Cargar un archivo Parquet
df = pd.read_parquet("C:\\Proyecto Final Henry\\DataBase\\processed\\Compras.parquet")

# Mostrar las primeras filas
print(df.head())

print(df.columns)

   ProveedorId              NombreProveedor FechaFactura  CompraId  \
0          105  ALTAMAR BRANDS LLC            2016-01-04      8124   
1         4466  AMERICAN VINTAGE BEVERAGE     2016-01-07      8137   
2          388  ATLANTIC IMPORTING COMPANY    2016-01-09      8169   
3          480  BACARDI USA INC               2016-01-12      8106   
4          516  BANFI PRODUCTS CORP           2016-01-07      8170   

  FechaCompra   FechaPago  Cantidad  CostoTotal  CostoEnvio  
0  2015-12-21  2016-02-16         6      214.26        3.47  
1  2015-12-22  2016-02-21        15      140.55        8.57  
2  2015-12-24  2016-02-16         5      106.60        4.61  
3  2015-12-20  2016-02-05     10100   137483.78     2935.20  
4  2015-12-24  2016-02-12      1935    15527.25      429.20  
Index(['ProveedorId', 'NombreProveedor', 'FechaFactura', 'CompraId',
       'FechaCompra', 'FechaPago', 'Cantidad', 'CostoTotal', 'CostoEnvio'],
      dtype='object')


### Subir archivo al bucket a la carpeta Datos Limpios

In [19]:
def subir_a_gcs(bucket_name, source_file_path4, folder_name, destination_file_name4):
    """Sube un archivo a una carpeta específica dentro de un bucket en Google Cloud Storage."""
    
    # Inicializar el cliente de GCS
    storage_client = storage.Client()
    
    # Obtener el bucket
    bucket = storage_client.bucket(bucket_name)
    
    # Definir la ruta completa en el bucket (carpeta + nombre de archivo)
    destination_blob_name = f"{folder_name}/{destination_file_name4}".strip("/")
    
    # Crear un blob (objeto en el bucket)
    blob = bucket.blob(destination_blob_name)
    
    # Subir el archivo
    blob.upload_from_filename(source_file_path4)

    print(f"\n🚀 Archivo {source_file_path4} subido a gs://{bucket_name}/{destination_blob_name}")

# Datos específicos de tu bucket y archivo
bucket_name = "licoreradatos"  # Tu bucket en GCS
folder_name = "Datos Limpios"  # Carpeta dentro del bucket
source_file_path4 = r"C:\\Proyecto Final Henry\\DataBase\\processed\\Compras.parquet"  # Archivo local
destination_file_name4 = "Compras.parquet"  # Nombre final en el bucket

# Subir archivo a GCS
subir_a_gcs(bucket_name, source_file_path4, folder_name, destination_file_name4)


🚀 Archivo C:\\Proyecto Final Henry\\DataBase\\processed\\Compras.parquet subido a gs://licoreradatos/Datos Limpios/Compras.parquet


## ETL archivo PurchasesFINAL12312016

In [20]:
def cargar_csv(ruta_csv5):
    """Carga un archivo CSV en un DataFrame de Pandas."""
    df5 = pd.read_csv(ruta_csv5)
    print(f"✅ Archivo cargado: {ruta_csv5}")
    return df5

def revisar_datos(df5):
    """Revisa el total de filas y columnas, valores faltantes y duplicados."""
    print("\n🔍 Revisión inicial de los datos:")
    print(f"Forma del DataFrame (filas, columnas): {df5.shape}")
    print("\nValores faltantes por columna:\n", df5.isnull().sum())
    
    duplicados = df5.duplicated().sum()
    print(f"\nTotal de filas duplicadas: {duplicados}")

def limpiar_datos(df5):
    """Elimina valores nulos en la columna 'Tamano' y filas duplicadas sin afectar el conteo final esperado."""
    # Verificar si la columna "Tamano" existe y corregir posibles errores en el nombre
    col_tamano = "Tamano" if "Tamano" in df5.columns else "Size" if "Size" in df5.columns else None

    if col_tamano:
        filas_size_nulas = df5[col_tamano].isnull().sum()
        df5 = df5.dropna(subset=[col_tamano])
        print(f"\n🚮 {filas_size_nulas} filas con valores NaN en '{col_tamano}' eliminadas.")

    print("\n✅ Datos limpiados correctamente.")
    return df5  

def renombrar_columnas(df):
    """Renombra las columnas del DataFrame según un formato estándar."""
    df5.columns = [
        "InventarioIncialId", "Tienda", "ProductoId", "NombreProducto", "Tamano",
        "ProveedorId", "NombreProveedor", "CompraId", "FechaCompra", "FechaIngreso",
        "FechaFactura", "FechaPago", "CostoUnitario", "Cantidad", "CostoTotal", "Clasificacion"
    ]
    print("\n📌 Columnas renombradas correctamente.")
    return df5  

def guardar_parquet(df5, output_path):
    """Guarda el DataFrame en formato Parquet con compresión Snappy."""
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df5.to_parquet(output_path, engine="pyarrow", compression="snappy")
    print(f"\n✅ Archivo guardado en Parquet: {output_path}")

# ------------------ 🔹 EJECUCIÓN DEL PROCESO ETL 🔹 ------------------
ruta_csv5 = r"C:\\Proyecto Final Henry\\DataBase\\PurchasesFINAL12312016.csv"
ruta_parquet5 = r"C:\\Proyecto Final Henry\\DataBase\\processed\\DetalleCompra.parquet"

# 1️⃣ Cargar datos
df5 = cargar_csv(ruta_csv5)

# 2️⃣ Revisar datos
revisar_datos(df5)

# 3️⃣ Limpiar datos (solo eliminamos valores nulos en 'Tamano')
df5 = limpiar_datos(df5)

# 4️⃣ Renombrar columnas
df5 = renombrar_columnas(df5)

# 5️⃣ Verificar el número de filas después de la limpieza
print(f"\n🔎 Filas finales esperadas: 2,372,471")
print(f"🔎 Filas actuales después de limpieza: {df5.shape[0]}")

# 6️⃣ Guardar en Parquet
guardar_parquet(df5, ruta_parquet5)

✅ Archivo cargado: C:\\Proyecto Final Henry\\DataBase\\PurchasesFINAL12312016.csv

🔍 Revisión inicial de los datos:
Forma del DataFrame (filas, columnas): (2372474, 16)

Valores faltantes por columna:
 InventoryId       0
Store             0
Brand             0
Description       0
Size              3
VendorNumber      0
VendorName        0
PONumber          0
PODate            0
ReceivingDate     0
InvoiceDate       0
PayDate           0
PurchasePrice     0
Quantity          0
Dollars           0
Classification    0
dtype: int64

Total de filas duplicadas: 0

🚮 3 filas con valores NaN en 'Size' eliminadas.

✅ Datos limpiados correctamente.

📌 Columnas renombradas correctamente.

🔎 Filas finales esperadas: 2,372,471
🔎 Filas actuales después de limpieza: 2372471

✅ Archivo guardado en Parquet: C:\\Proyecto Final Henry\\DataBase\\processed\\DetalleCompra.parquet


In [21]:
import pandas as pd

# Cargar un archivo Parquet
df = pd.read_parquet("C:\\Proyecto Final Henry\\DataBase\\processed\\DetalleCompra.parquet")

# Mostrar las primeras filas
print(df.head())

print(df.columns)

    InventarioIncialId  Tienda  ProductoId                NombreProducto  \
0    69_MOUNTMEND_8412      69        8412     Tequila Ocho Plata Fresno   
1     30_CULCHETH_5255      30        5255  TGI Fridays Ultimte Mudslide   
2    34_PITMERDEN_5215      34        5215  TGI Fridays Long Island Iced   
3  1_HARDERSFIELD_5255       1        5255  TGI Fridays Ultimte Mudslide   
4    76_DONCASTER_2034      76        2034     Glendalough Double Barrel   

  Tamano  ProveedorId              NombreProveedor  CompraId FechaCompra  \
0  750mL          105  ALTAMAR BRANDS LLC               8124  2015-12-21   
1  1.75L         4466  AMERICAN VINTAGE BEVERAGE        8137  2015-12-22   
2  1.75L         4466  AMERICAN VINTAGE BEVERAGE        8137  2015-12-22   
3  1.75L         4466  AMERICAN VINTAGE BEVERAGE        8137  2015-12-22   
4  750mL          388  ATLANTIC IMPORTING COMPANY       8169  2015-12-24   

  FechaIngreso FechaFactura   FechaPago  CostoUnitario  Cantidad  CostoTotal  \
0   20

### Subir archivo al bucket a la carpeta Datos Limpios

In [22]:
def subir_a_gcs(bucket_name, source_file_path5, folder_name, destination_file_name5):
    """Sube un archivo a una carpeta específica dentro de un bucket en Google Cloud Storage."""
    
    # Inicializar el cliente de GCS
    storage_client = storage.Client()
    
    # Obtener el bucket
    bucket = storage_client.bucket(bucket_name)
    
    # Definir la ruta completa en el bucket (carpeta + nombre de archivo)
    destination_blob_name = f"{folder_name}/{destination_file_name5}".strip("/")
    
    # Crear un blob (objeto en el bucket)
    blob = bucket.blob(destination_blob_name)
    
    # Subir el archivo
    blob.upload_from_filename(source_file_path5)

    print(f"\n🚀 Archivo {source_file_path5} subido a gs://{bucket_name}/{destination_blob_name}")

# Datos específicos de tu bucket y archivo
bucket_name = "licoreradatos"  # Tu bucket en GCS
folder_name = "Datos Limpios"  # Carpeta dentro del bucket
source_file_path5 = r"C:\\Proyecto Final Henry\\DataBase\\processed\\DetalleCompra.parquet"  # Archivo local
destination_file_name5 = "DetalleCompra.parquet"  # Nombre final en el bucket

# Subir archivo a GCS
subir_a_gcs(bucket_name, source_file_path5, folder_name, destination_file_name5)


🚀 Archivo C:\\Proyecto Final Henry\\DataBase\\processed\\DetalleCompra.parquet subido a gs://licoreradatos/Datos Limpios/DetalleCompra.parquet


## ETL archivo SalesFINAL12312016

In [23]:

def cargar_csv(ruta_csv6):
    """Carga un archivo CSV en un DataFrame de Pandas."""
    df6 = pd.read_csv(ruta_csv6)
    print(f"✅ Archivo cargado: {ruta_csv6}")
    return df6

def revisar_datos(df6):
    """Revisa el total de filas y columnas, valores faltantes y duplicados."""
    print("\n🔍 Revisión inicial de los datos:")
    print(f"Forma del DataFrame (filas, columnas): {df6.shape}")
    print("\nValores faltantes por columna:\n", df6.isnull().sum())

    duplicados = df6.duplicated().sum()
    print(f"\nTotal de filas duplicadas: {duplicados}")

def limpiar_datos(df6):
    """Elimina valores nulos y filas duplicadas para limpiar la base de datos."""
    # Eliminar filas con valores nulos
    filas_nulas_totales = df6.isnull().any(axis=1).sum()
    df6 = df6.dropna()
    print(f"\n🚮 {filas_nulas_totales} filas con valores nulos eliminadas.")

    # Eliminar duplicados
    filas_duplicadas = df6.duplicated().sum()
    df6 = df6.drop_duplicates()
    print(f"\n🚮 {filas_duplicadas} filas duplicadas eliminadas.")

    print("\n✅ Datos limpiados correctamente.")
    return df6  

def renombrar_columnas(df6):
    """Renombra las columnas del DataFrame según un formato estándar."""
    df6.columns = [
        "InventarioInicialId", "Tienda", "ProductoId", "NombreProducto", "Tamano",
        "CantidadVendida", "TotalVenta", "PrecioUnitario", "FechaVenta", "Volumen",
        "Clasificacion", "Impuestos", "ProveedorId", "NombreProveedor", "Costo"
    ]
    print("\n📌 Columnas renombradas correctamente.")
    return df6  

def guardar_parquet(df6, output_path):
    """Guarda el DataFrame en formato Parquet con compresión Snappy."""
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df6.to_parquet(output_path, engine="pyarrow", compression="snappy")
    print(f"\n✅ Archivo guardado en Parquet: {output_path}")


# ------------------ 🔹 EJECUCIÓN DEL PROCESO ETL 🔹 ------------------
ruta_csv6 = r"C:\\Proyecto Final Henry\\DataBase\\SalesFINAL123120161.csv"
ruta_parquet6 = r"C:\\Proyecto Final Henry\\DataBase\\processed\\Ventas.parquet"

# 1️⃣ Cargar datos
df6 = cargar_csv(ruta_csv6)

# 2️⃣ Revisar datos
revisar_datos(df6)

# 3️⃣ Limpiar datos
df6 = limpiar_datos(df6)

# 4️⃣ Renombrar columnas
df6 = renombrar_columnas(df6)

# 5️⃣ Verificar el número de filas después de la limpieza
print(f"\n🔎 Filas finales después de limpieza: {df6.shape[0]}")

# 6️⃣ Guardar en Parquet
guardar_parquet(df6, ruta_parquet6)



✅ Archivo cargado: C:\\Proyecto Final Henry\\DataBase\\SalesFINAL123120161.csv

🔍 Revisión inicial de los datos:
Forma del DataFrame (filas, columnas): (1048575, 15)

Valores faltantes por columna:
 InventoryId       0
Store             0
Brand             0
Description       0
Size              0
SalesQuantity     0
SalesDollars      0
SalesPrice        0
SalesDate         0
Volume            0
Classification    0
ExciseTax         0
VendorNo          0
VendorName        0
PurchasePrice     0
dtype: int64

Total de filas duplicadas: 0

🚮 0 filas con valores nulos eliminadas.

🚮 0 filas duplicadas eliminadas.

✅ Datos limpiados correctamente.

📌 Columnas renombradas correctamente.

🔎 Filas finales después de limpieza: 1048575

✅ Archivo guardado en Parquet: C:\\Proyecto Final Henry\\DataBase\\processed\\Ventas.parquet


In [24]:
# Cargar un archivo Parquet
df = pd.read_parquet("C:\\Proyecto Final Henry\\DataBase\\processed\\Ventas.parquet")

# Mostrar las primeras filas
print(df.head())

print(df.columns)

   InventarioInicialId  Tienda  ProductoId              NombreProducto  \
0  1_HARDERSFIELD_1004       1        1004  Jim Beam w/2 Rocks Glasses   
1  1_HARDERSFIELD_1004       1        1004  Jim Beam w/2 Rocks Glasses   
2  1_HARDERSFIELD_1004       1        1004  Jim Beam w/2 Rocks Glasses   
3  1_HARDERSFIELD_1004       1        1004  Jim Beam w/2 Rocks Glasses   
4  1_HARDERSFIELD_1005       1        1005     Maker's Mark Combo Pack   

       Tamano  CantidadVendida  TotalVenta  PrecioUnitario FechaVenta  \
0       750mL                1       16.49           16.49   1/1/2016   
1       750mL                2       32.98           16.49   1/2/2016   
2       750mL                1       16.49           16.49   1/3/2016   
3       750mL                1       14.49           14.49   1/8/2016   
4  375mL 2 Pk                2       69.98           34.99   1/9/2016   

   Volumen  Clasificacion  Impuestos  ProveedorId  \
0      750              1       0.79        12546   
1      750

### Subir archivo al bucket a la carpeta Datos Limpios

In [25]:
def subir_a_gcs(bucket_name, source_file_path6, folder_name, destination_file_name6):
    """Sube un archivo a una carpeta específica dentro de un bucket en Google Cloud Storage."""
    
    # Inicializar el cliente de GCS
    storage_client = storage.Client()
    
    # Obtener el bucket
    bucket = storage_client.bucket(bucket_name)
    
    # Definir la ruta completa en el bucket (carpeta + nombre de archivo)
    destination_blob_name = f"{folder_name}/{destination_file_name6}".strip("/")
    
    # Crear un blob (objeto en el bucket)
    blob = bucket.blob(destination_blob_name)
    
    # Subir el archivo
    blob.upload_from_filename(source_file_path6)

    print(f"\n🚀 Archivo {source_file_path6} subido a gs://{bucket_name}/{destination_blob_name}")

# Datos específicos de tu bucket y archivo
bucket_name = "licoreradatos"  # Tu bucket en GCS
folder_name = "Datos Limpios"  # Carpeta dentro del bucket
source_file_path6 = r"C:\\Proyecto Final Henry\\DataBase\\processed\\Ventas.parquet"  # Archivo local
destination_file_name6 = "Ventas.parquet"  # Nombre final en el bucket

# Subir archivo a GCS
subir_a_gcs(bucket_name, source_file_path6, folder_name, destination_file_name6)


🚀 Archivo C:\\Proyecto Final Henry\\DataBase\\processed\\Ventas.parquet subido a gs://licoreradatos/Datos Limpios/Ventas.parquet
